In [1]:
from interfacemaster.cellcalc import get_primitive_hkl, get_pri_vec_inplane, get_normal_index, get_normal_from_MI, rot
from interfacemaster.interface_generator import core, print_near_axis, convert_vector_index, get_disorientation
from numpy import array, dot, round, cross, eye
from numpy.linalg import inv, det, norm

In [5]:
my_interface = core('cif_files/GaN_mp-804_conventional_standard (1).cif',\
                    'cif_files/Fe_mp-13_conventional_standard.cif')

Warning!, this programme will rewrite the POSCAR file in this dir!


In [6]:
R = get_disorientation(L1 = my_interface.conv_lattice_1, L2 = my_interface.conv_lattice_2, \
                       v1 = [-1,-1,0], hkl1 = [1, -1, 0], \
                       v2 = [0,-1,0], hkl2 = [2, 0, 5])
R

array([[ 3.21633760e-01,  5.00000000e-01,  8.04084401e-01],
       [-1.85695338e-01,  8.66025404e-01, -4.64238345e-01],
       [-9.28476691e-01,  4.28707594e-17,  3.71390676e-01]])

# Get the two axis aligned to be collinear, expressed in primitive cell

Here, we want the (1,-1,0,2) plane of the Al2O3 coincident whith (1,1,-2,0) plane of ZnO.

Firstly, the programme rotate lattice 2 so that the two lattice plane overlap.

As the normal of the plane have different indices with the miller indices, we need to compute it.

In [7]:
my_interface.conv_lattice_2

array([[ 2.84005168e+00, -1.73903010e-16,  0.00000000e+00],
       [ 0.00000000e+00,  2.84005168e+00,  0.00000000e+00],
       [ 1.73903010e-16,  1.73903010e-16,  2.84005168e+00]])

In [8]:
hkl_1 = get_primitive_hkl([1, -1, 0], my_interface.conv_lattice_1, my_interface.lattice_1)
hkl_2 = get_primitive_hkl([2, 0, 5], my_interface.conv_lattice_2, my_interface.lattice_2)
v2 = dot(my_interface.conv_lattice_2,[0,-1,0])
v2

array([ 1.73903010e-16, -2.84005168e+00, -1.73903010e-16])

In [9]:
my_interface.conv_lattice_1

array([[ 3.21629007e+00, -1.60814503e+00,  0.00000000e+00],
       [ 0.00000000e+00,  2.78538891e+00,  0.00000000e+00],
       [ 1.96940967e-16,  1.96940967e-16,  5.23996200e+00]])

In [12]:
n1 = get_normal_from_MI(my_interface.lattice_1,hkl_1)
n1

array([14.59533203, -8.42661887,  0.        ])

In [9]:
dot(R,v2)/norm(dot(R,v2))

array([-5.00000000e-01, -8.66025404e-01, -1.22464680e-16])

# Searching two_dimensional CSL

In [13]:
my_interface.parse_limit(du = 1e-1, S  = 1e-1, sgm1=100, sgm2=100, dd = 1e-1)

#Do searching!
my_interface.search_one_position_2D(hkl_1, hkl_2, theta_range = 180, \
                                    dtheta = 0.01, exact_R = R, match_tol = 0.1, pre_dt= True)

Congrates, we found an appx CSL!

U1 = 
[[4 0]
 [0 0]
 [0 3]]; sigma_1 = 12

U2 = 
[[-5  5]
 [ 0  7]
 [-5  2]]; sigma_2 = 52

D = 
[[ 0.97649518 -0.04071154  0.        ]
 [-0.04071154  0.92948554  0.        ]
 [-0.         -0.          1.02783677]]

axis = [14.59533203 -8.42661887  0.        ] ; theta = 0.0



# Get bicrystal

In [14]:
R

array([[ 3.21633760e-01,  5.00000000e-01,  8.04084401e-01],
       [-1.85695338e-01,  8.66025404e-01, -4.64238345e-01],
       [-9.28476691e-01,  4.28707594e-17,  3.71390676e-01]])

In [15]:
my_interface.a2_transform

array([[ 3.21633760e-01,  4.52990358e-01,  8.04084401e-01],
       [-1.85695338e-01,  7.84602316e-01, -4.64238345e-01],
       [-9.54322483e-01, -1.32160127e-15,  3.81728993e-01]])

In [16]:
my_interface.compute_bicrystal_two_D(hkl_1 = hkl_1, hkl_2=hkl_2, normal_ortho = True, plane_ortho = False, lim = 50, tol_ortho = 1e-2, tol_integer=1e-3)

cell 1:
[[-1  4  0]
 [-2  0  0]
 [ 0  0  3]]
cell 2:
[[ 1 -5  5]
 [-1  0  7]
 [-2 -5  2]]


In [17]:
my_interface.get_bicrystal(two_D = True, xyz_1 = [1,1,1], xyz_2 = [1,1,1])

Please check the POSCAR file